# analyzing XML sent from CsPy to LabVIEW server 
(and potentially vice versa)

In [1]:
# built-in 
import re
import xml.etree.ElementTree as ET
from ctypes import *

# TODO: create requirements.txt for these
from recordclass import recordclass as rc # for Trigger, Waveform types
import numpy as np # for arrays
from ctypes import c_uint32

In [2]:
# to_pxi is a text file containing what is sent to the PXI from CsPy when 
# 'update settings' is clicked. 

# Each message is separated from the last by "NEW MESSAGE" \n (added to 
# file as a message delimiter; not part of the xml string sent)

# separate the text file into chunks of texts for each time
# LabVIEW.send was called
def msg_from_file(file="to_pxi.txt"): # 
    msgs = []
    with open(file) as f:
        lines = f.readlines()
        for line in lines:
            if "NEW MESSAGE" in line:
                msgs.append('')
                continue
            msgs[-1] += line
    return msgs

def xml_to_dict(xml, deep=True):
    """
    'xml': str containing xml to parse, or xml.etree.ElementTree.Element
    'deep': if True, search children of children
    """
    xml_dict = {}
    if type(xml) == str:
        root = ET.fromstring(xml)
    elif type(xml) == ET.Element:
        root = xml
    else: 
        print("not a valid input format")
        
    for child in root:
        xml_dict[child.tag] = {
            'text': child.text,
            'tail': child.tail,
            'children': None
        }
        
            
        if deep and list(child):
            xml_dict[child.tag]['children'] = xml_to_dict(child)

    return xml_dict

def print_txt(node):
    print(f"{node.tag} = {node.text}")

### Get the root from an XML string

In [3]:
msg = msg_from_file()[0]
root = ET.fromstring(msg)
# type(root) == ET.Element
# list(root)
root

<Element 'LabView' at 0x0BDCFB68>

### Get a dict from the XML string

In [4]:
# the "update settings" xml, suppressing grandchildren
xml_to_dict(msg, deep=False)

{'timeout': {'text': '15.0', 'tail': '\n', 'children': None},
 'AnalogOutput': {'text': None, 'tail': '\n', 'children': None},
 'AnalogInput': {'text': None, 'tail': '\n', 'children': None},
 'HSDIO': {'text': None, 'tail': '\n', 'children': None},
 'piezo': {'text': None, 'tail': None, 'children': None},
 'RF_generators': {'text': None, 'tail': None, 'children': None},
 'DAQmxDO': {'text': None, 'tail': None, 'children': None},
 'camera': {'text': None, 'tail': None, 'children': None},
 'TTL': {'text': None, 'tail': '\n', 'children': None},
 'Counters': {'text': None, 'tail': None, 'children': None},
 'cycleContinuously': {'text': 'True', 'tail': '\n', 'children': None}}

In [78]:
# this does not suppress grandchildren. 
# warning: the waveform text is long!
update_dict = xml_to_dict(msg) 
# update_dict

### Look at specific elements:

In [79]:
update_dict['HSDIO']['children']

{'script': {'text': 'script script1\ngenerate c1\nwait 9990\ngenerate w0xb6ba30000000040L\nwait 40000\ngenerate w0x3eba30000020040L\nwait 50000\ngenerate w0x3eba30000020040L\nwait 40000\ngenerate w0x3ebb30000020040L\nwait 10000\ngenerate w0x3eba30000000040L\nwait 50000\ngenerate w0x3eba30000000040L\nwait 50000\ngenerate w0xb6ba30000000040L\nwait 750000\ngenerate w0xb6ba78200008040L\nwait 210000\ngenerate w0xb6ba78200008000L\nwait 90000\ngenerate w0xb6ba30000008000L\nwait 100000\ngenerate w0xb6ba38000008000L\nwait 100000\ngenerate w0xb6bb38000000000L\nwait 100000\ngenerate w0xb6ba30000000000L\nwait 4400000\ngenerate w0x1beba30000000000L\nwait 3500000\ngenerate w0x1bebab0000000000L\nwait 500000\ngenerate w0x3faaa30000000000L\nwait 3000000\ngenerate c2\nwait 999990\ngenerate c3\nwait 12\nrepeat 1249\ngenerate c4\nend repeat\ngenerate c5\nrepeat 4999\ngenerate c6\nend repeat\ngenerate c7\ngenerate c8\ngenerate c9\ngenerate c10\nwait 82\ngenerate w0x66aaa30000000000L\nwait 99900\ngenerate c

## Reproduce XML parsing for the HSDIO
 * Get the root from a string
 * cycle through children to find the HSDIO tag
 * mimic the LabVIEW code for dealing with this tag
 * ultimately have string ready to be passed as input to the functions for triggering and writing to PC memory
 
### TODO: 
 * once i have code to fully parse string for HSDIO, build the methods into hsdio.py so this can be imported as a module testing, e.g. so Juan could get args to pass to c functions
 * use array or numpy array types in place of python lists where applicable

In [5]:
# built-in 
import re
import xml.etree.ElementTree as ET
from ctypes import *

# TODO: create requirements.txt for these
from recordclass import recordclass as rc # for Trigger, Waveform types
import numpy as np # for arrays
from ctypes import c_uint32

# local module imports
from trigger import Trigger, StartTrigger
from waveform import Waveform
from hsdio import HSDIO

# build update string from text file
msg = msg_from_file()[0] 

In [6]:
# find the root:
root = ET.fromstring(msg)
print(root)
if root.tag != "LabView":
    print("Not a valid msg for the pxi")

<Element 'LabView' at 0x0AFE70C8>


AttributeError: 'HSDIO' object has no attribute 'dllpath32'

In [12]:
for child in root:
    if child.tag == "HSDIO":
        
#         hsdio = HSDIO()
        # wrap this block in HSDIO.load_xml
        hsdio_nodes = child
        for child in hsdio_nodes:
            
            # the LabView code ignores non-element nodes. not sure if this equivalent
            if type(child) == ET.Element:
                
                # handle each tag by name:
                #if child.tag == "version": # looks like this hasn't been used since 2015

                if child.tag == "enable":
                    print_txt(child)
                    #hsdio.enablePulses = bool(child.text)
                    pass
                
                elif child.tag == "description":
                    print_txt(child)
                    #hsdio.description = child.text
                    pass
                
                elif child.tag == "resourceName":
                    print_txt(child)
                    resources = child.text.split(",")
                    #hsdio.resourceNames = resources
                    
                elif child.tag == "clockRate":
                    clockRate = float(child.text)
                    print_txt(child)
                    #hsdio.clockRate = clockRate
                elif child.tag == "hardwareAlignmentQuantum":
                    print_txt(child)
                    #hsdio.hardwareAlignmentQuantum = child.text
                    pass  
                
                elif child.tag == "triggers":
                    print_txt(child)
                    
                    if type(child) == ET.Element:
                        
                        trigger_node = child
                        
                        # for each line of script triggers
                        for child in trigger_node:
                            if type(child) == ET.Element:
                                
                                trig = Trigger()
                                trig.init_from_xml(child)
                                  
#                               hsdio.scriptTriggerArr.append(trig)
                            
                elif child.tag == "waveforms":

                    #print_txt(child) # HUGE STRING PLZ BE CAREFUL
                    print("found waveforms") #TODO: change to logger
    
                    # TODO: wrap in load waveform xml
                    wvforms_node = child
    
                    # for each waveform
                    for wvf_child in wvforms_node:
                
                        if type(wvf_child) == ET.Element:
                                                                                    
                            if wvf_child.tag == "waveform":

                                wvform = Waveform()
                                wvform.init_from_xml(wvf_child)
                                #hsdio.waveformArr.append(wvform)

                elif child.tag == "script":
                    print_txt(child)
                    #hsdio.pulseGenScript = child.text
                    pass
            
                elif child.tag == "startTrigger":
                    print_txt(child)
                                        
#                     hsdio.startTrigger = StartTrigger()
#                   hsdio.startTrigger = init_from_xml(child)
                    startTrigger = StartTrigger()
                    startTrigger.init_from_xml(child)
                    
                
                elif child.tag == "InitialState":
                    print_txt(child)
                    pass
                
                elif child.tag == "IdleState":
                    print_txt(child)
                    pass      
                
                elif child.tag == "ActiveChannels":
                    print_txt(child)
                    activeChannels = np.array(child.text.split("\n"))
                    print(f"active channels={activeChannels}")
                
                else:
                    pass

script = script script1
generate c1
wait 9990
generate w0xb6ba30000000040L
wait 40000
generate w0x3eba30000020040L
wait 50000
generate w0x3eba30000020040L
wait 40000
generate w0x3ebb30000020040L
wait 10000
generate w0x3eba30000000040L
wait 50000
generate w0x3eba30000000040L
wait 50000
generate w0xb6ba30000000040L
wait 750000
generate w0xb6ba78200008040L
wait 210000
generate w0xb6ba78200008000L
wait 90000
generate w0xb6ba30000008000L
wait 100000
generate w0xb6ba38000008000L
wait 100000
generate w0xb6bb38000000000L
wait 100000
generate w0xb6ba30000000000L
wait 4400000
generate w0x1beba30000000000L
wait 3500000
generate w0x1bebab0000000000L
wait 500000
generate w0x3faaa30000000000L
wait 3000000
generate c2
wait 999990
generate c3
wait 12
repeat 1249
generate c4
end repeat
generate c5
repeat 4999
generate c6
end repeat
generate c7
generate c8
generate c9
generate c10
wait 82
generate w0x66aaa30000000000L
wait 99900
generate c11
wait 38
repeat 1999
generate c12
end repeat
generate c13
repea

In [17]:
startTrigger

StartTrigger(waitForStartTrigger=False, source=, description=, edge=c_ulong(0))

In [71]:
len("0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0".split(" "))

64

In [70]:
states = "0 1 0 1 0 0"''
np.array([x for x in states.split(" ")], dtype=c_uint32)

array([0, 1, 0, 1, 0, 0], dtype=uint32)

In [34]:
c_uint32(10)

c_ulong(10)

In [73]:
text = """0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0"""

In [90]:
lines = text.split("\n")
# arr = np.empty((len(lines), 64))
               
[[c_uint32(int(x)) for x in line.split(" ")] for line in lines]

[[c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(1),
  c_ulong(0),
  c_ulong(0),
  c_ulong(1),
  c_ulong(0),
  c_ulong(1),
  c_ulong(1),
  c_ulong(0),
  c_ulong(1),
  c_ulong(0),
  c_ulong(1),
  c_ulong(1),
  c_ulong(1),
  c_ulong(0),
  c_ulong(1),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(1),
  c_ulong(1),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0)],
 [c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(0),
  c_ulong(1),
  c_ulong(0),
  c_ulong(0),
  c_u

In [93]:
lines = text.split("\n")

In [105]:
np.array([[int(x) for x in line.split(" ")] for line in lines], dtype=c_uint32)

array([[0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=uint32)

In [13]:
class bar:
    if True:
        prop = [1,2,3]
    else: 
        prop = []
    def __init__(self):
        self.init = True
        print("bar initialized!")

class foo(bar):
    def __init__(self):
        self.x = x
        print("foo initialized!")

In [14]:
bar.prop

[1, 2, 3]

In [23]:
f.prop

[1, 2, 3]

In [21]:
"True".lower() == "true"

True

In [9]:
np.array("a,b,c,d".split(","),dtype=str)

array(['a', 'b', 'c', 'd'], dtype='<U1')

In [15]:
x = "hi"
y = f"oh {x} there"
y

'oh hi there'

In [16]:
zip([1,2], [1,2,3])

In [17]:
for a,b in zip([1,2], [1,2,3]):
    print(a,b)

1 1
2 2
